In [1]:
%load_ext autoreload
%autoreload 2
from pyMSOO.MFEA.model import MFEA_base
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 
import pickle

In [2]:
from pyMSOO.MFEA.benchmark.continous.funcs import Griewank

In [8]:
dim = 2
griewank = Griewank(dim)
bound = 50
x = np.array([1, 1])

In [9]:
griewank(x)

[1 2]
0.0005


0.5897380911762422

In [32]:
x

array([1, 1])

In [20]:
griewank(x)

0.5897380911762422

In [2]:
with open('x.pkl', 'wb') as f:  # open a text file
    pickle.dump(x, f) # serialize the list
f.close()

NameError: name 'x' is not defined

In [14]:
from pyMSOO.MFEA.benchmark.continous.funcs import Ackley

In [15]:
super_tasks = CEC17_benchmark.get_10tasks_benchmark()

In [3]:
with open('x_test.pkl','rb') as f:
    x_test = pickle.load(f)

print(x_test)

[0.85907482 0.94008339 0.50785754 0.72509466 0.06625432 0.66633294
 0.74234832 0.90374985 0.23927356 0.32128177 0.31552997 0.67909715
 0.43332934 0.98120044 0.2745511  0.60376838 0.85184482 0.19065958
 0.6426098  0.40787485 0.20654012 0.13219135 0.41308363 0.54744061
 0.57048436 0.42555876 0.19406744 0.27179178 0.297762   0.11140318
 0.45006828 0.16531338 0.92069788 0.88879073 0.44608596 0.23528611
 0.58133187 0.71235906 0.17826929 0.45866792 0.01148249 0.91859941
 0.35862434 0.36346905 0.90740653 0.8934829  0.99067995 0.27788791
 0.08522302 0.15000237]


In [4]:
from pyMSOO.MFEA.benchmark.continous.funcs import *

In [5]:
tasks = [
        Sphere(     50,shift= 0,    bound= [-100, 100]),   # 0
        Sphere(     50,shift= 80,   bound= [-100, 100]),  # 80
        Sphere(     50,shift= -80,  bound= [-100, 100]), # -80
        Weierstrass(25,shift= -0.4, bound= [-0.5, 0.5]), # -0.4
        Rosenbrock( 50,shift= -1,   bound= [-50, 50]),# 0
        Ackley(     50,shift= 40,   bound= [-50, 50]),    # 40
        Weierstrass(50,shift= -0.4, bound= [-0.5, 0.5]), # -0.4
        Schwefel(   50,shift= 0,    bound= [-500, 500], fixed = False), # 420.9687
        Griewank(   50,shift= [-80, 80], bound= [-100, 100]), # -80, 80
        Rastrigin(  50,shift= [40, -40], bound= [-50, 50]),# -40, 40
        ]

In [6]:
for task in tasks:
    print(task(x_test))

165305.0500415824
500797.2655241336
469812.83455903106
52.41419772668738
5980077333.781868
21.637249102506384
96.75042619986894
18339.884641225744
137.74422260305982
106369.76081990033


[164653.77602241]
[506484.7290924]
[462822.82295242]
[51.62977499]
[5.66607615e+09]
[21.61071135]
[99.04091139]
[21553.29522811]
[109.39305966]
[134383.12832027]